# Setup

In [ ]:
import os
import logging as log
from lisa.utils import setup_logging
setup_logging(level=log.ERROR)

from lisa.trace import Trace
from lisa.wa import WAOutput
from lisa.stats import Stats
from lisa.datautils import series_mean
from pandas import DataFrame
import pandas as pd
import scipy as sp
import numpy as np
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go
import holoviews as hv
from holoviews import opts
from bokeh.themes import built_in_themes
from tabulate import tabulate

from wp.notebook import WorkloadNotebookAnalysis, trim_wa_path, ptable, trim_number


hv.extension('bokeh')
hv.renderer('bokeh').theme = built_in_themes['dark_minimal']
hv.renderer('bokeh').webgl = True
pio.templates.default = "plotly"
pio.templates.default = "plotly_dark"

color_cycle = hv.Cycle(['#636EFA', '#EF553B', '#00CC96', '#AB63FA', '#FFA15A', '#19D3F3', '#FF6692', '#B6E880', '#FF97FF', '#FECB52'])

opts.defaults(
    opts.Curve(tools=['hover'], show_grid=True, color=color_cycle, muted_alpha=0),
    opts.Table(bgcolor='black')
)

## Runs

In [ ]:
gb5 = WorkloadNotebookAnalysis('/home/kajpuc01/power/pixel6/geekbench/', [
    'geekbench_product_baseline_3_1407',
    'geekbench_product_arm_vh_sched_debug_3_1907',
])

gb5.show()

# Benchmark scores

## Line plot

In [ ]:
metrics = ['multicore_score', 'score']
ds = hv.Dataset(gb5.results, ['iteration', hv.Dimension('wa_path', values=gb5.wa_paths), hv.Dimension('metric', values=metrics)], 'value')
layout = ds.select(metric=metrics).to(hv.Curve, 'iteration', 'value').overlay('wa_path').opts(legend_position='bottom').layout('metric').opts(shared_axes=False, title='Benchmark score per-iteration').cols(2)
layout.opts(
    opts.Curve(height=600, width=900),
)
layout

## Bar plot

In [ ]:
gb5.summary['scores'] = gb5.plot_gmean_bars(gb5.results.query("metric in @metrics"), x='stat', y='value', facet_col='metric', facet_col_wrap=3, title='gmean benchmark score', width=1600, height=600)

# Overutilized

In [ ]:
def postprocess_overutil(df):
    df['time'] = round(df['time'], 2)
    df['total_time'] = round(df['total_time'], 2)
    return df

gb5.load_combined_analysis('overutilized.pqt', postprocess=postprocess_overutil)
gb5.load_combined_analysis('overutilized_mean.pqt')
ptable(gb5.analysis['overutilized_mean'])

## Line plot

In [ ]:
gb5.plot_lines_px(gb5.analysis['overutilized'], y='percentage', title='Overutilized percentage per-iteration')

# Perf

## Line plot

In [ ]:
metrics = ['cpu-migrations', 'context-switches', 'stalled-cycles-backend', 'page-faults', 'major-faults', 'cache-misses', 'instructions', 'cpu-cycles', 'cpu-clock']
ds = hv.Dataset(gb5.results_perf, ['iteration', hv.Dimension('wa_path', values=gb5.wa_paths), hv.Dimension('metric', values=metrics)], 'value')
layout = ds.select(metric=metrics).to(hv.Curve, 'iteration', 'value').overlay('wa_path').opts(legend_position='bottom').layout('metric').opts(shared_axes=False, title='Perf counters').cols(3)
layout.opts(
    opts.Curve(width=600, height=340),
    opts.Overlay(legend_position='bottom'),
)
layout

## Bar plot

In [ ]:
metrics = ['cpu-migrations', 'context-switches', 'stalled-cycles-backend', 'page-faults', 'major-faults', 'minor-faults', 'cache-misses', 'instructions', 'cpu-cycles', 'cpu-clock']
gb5.plot_gmean_bars(gb5.results_perf.query("metric in @metrics"), x='stat', y='value', facet_col='metric', facet_col_wrap=5, title='gmean perf counters', width=1900, height=900)

# Idle residency

In [ ]:
gb5.load_combined_analysis('idle_residency.pqt')
gb5.analysis['idle_residency'] = gb5.analysis['idle_residency'].groupby(['wa_path', 'cluster', 'idle_state'], sort=False).mean(numeric_only=True).reset_index()[['wa_path', 'cluster', 'idle_state', 'time']]
gb5.analysis['idle_residency']['time'] = round(gb5.analysis['idle_residency']['time'], 2)

display(gb5.analysis['idle_residency'])

## Bar plot

In [ ]:
fig = px.bar(gb5.analysis['idle_residency'], x='idle_state', y='time', color='wa_path', facet_col='cluster', barmode='group', text=gb5.analysis['idle_residency']['time'], width=1800, height=600, title='Idle state residencies')
fig.update_traces(textposition='outside')
fig.show(renderer='iframe')

# Idle misses

In [ ]:
def load_idle_misses(analysis):
    def load_idle_miss_df(analysis_dir):
        try:
            wakeup_count = len(pd.read_parquet(os.path.join(analysis_dir, 'cpu_idle.pqt')).query("state == -1"))
            misses = pd.read_parquet(os.path.join(analysis_dir, 'cpu_idle_miss_counts.pqt')).groupby(['wa_path', 'kernel', 'cluster', 'below']).sum().reset_index()
            misses['count_perc'] = round(misses['count'] / wakeup_count * 100, 3)
            return misses
        except FileNotFoundError as e:
            print(e)

    return pd.concat([load_idle_miss_df(os.path.join(analysis.benchmark_path, benchmark_dir, 'analysis')) for benchmark_dir in analysis.benchmark_dirs])


gb5.analysis['cpu_idle_misses'] = load_idle_misses(gb5)
gb5.analysis['cpu_idle_misses']['type'] = gb5.analysis['cpu_idle_misses']['below'].replace(0, 'too deep').replace(1, 'too shallow')
gb5.analysis['cpu_idle_misses']['wa_path'] = gb5.analysis['cpu_idle_misses']['wa_path'].map(trim_wa_path)

display(gb5.analysis['cpu_idle_misses'])

## Bar plot

In [ ]:
ptable(gb5.analysis['cpu_idle_misses'].groupby(['wa_path', 'type']).sum().reset_index()[['wa_path', 'type', 'count_perc']])
fig = px.bar(gb5.analysis['cpu_idle_misses'], x='type', y='count_perc', color='wa_path', facet_col='cluster', barmode='group', text=gb5.analysis['cpu_idle_misses']['count_perc'], width=1800, height=600, title='CPUIdle misses as percentage of all wakeups')
fig.show(renderer='iframe')

# Power usage

In [ ]:
def postprocess_pixel6_emeter_means(df):
    df_total = df.groupby(['wa_path', 'kernel', 'iteration']).sum(numeric_only=True).reset_index()
    df_total['channel'] = 'Total'

    df_cpu_total = df.query("channel.str.startswith('CPU')").groupby(['wa_path', 'kernel', 'iteration']).sum(numeric_only=True).reset_index()
    df_cpu_total['channel'] = 'CPU'
    return pd.concat([df, df_cpu_total, df_total])[['wa_path', 'kernel', 'iteration', 'channel', 'power']]



gb5.load_combined_analysis('pixel6_emeter.pqt')
gb5.load_combined_analysis('pixel6_emeter_mean.pqt', postprocess=postprocess_pixel6_emeter_means)

display(gb5.analysis['pixel6_emeter_mean'].head())

## Line plot

In [ ]:
gb5.plot_lines_px(gb5.analysis['pixel6_emeter_mean'], y='power', facet_col='channel', facet_col_wrap=3, height=1000, title='Mean power usage across iterations [mW]')

## Bar plot

In [ ]:
gb5.summary['power_usage'] = gb5.plot_gmean_bars(gb5.analysis['pixel6_emeter_mean'].rename(columns={'power':'value'}), x='channel', y='value', facet_col='metric', facet_col_wrap=5, title='Gmean power usage [mW]', width=1800, height=600, include_total=True, include_columns=['channel'])

# Energy estimate

In [ ]:
gb5.load_combined_analysis('energy_estimate_mean.pqt')
gb5.analysis['energy_estimate_melt'] = pd.melt(gb5.analysis['energy_estimate_mean'], id_vars=['iteration', 'wa_path'], value_vars=['little', 'mid', 'big', 'total']).rename(columns={'variable':'cluster'})

## Line plot

In [ ]:
gb5.plot_lines_px(gb5.analysis['energy_estimate_melt'], facet_col='cluster', height=1000, title='Mean energy estimate across iterations [bW]')

## Bar plot

In [ ]:
gb5.summary['energy_estimate'] = gb5.plot_gmean_bars(gb5.analysis['energy_estimate_melt'], x='cluster', y='value', facet_col='metric', facet_col_wrap=5, title='Gmean energy estimate [bW]', width=1900, height=600, include_columns=['cluster'], order_cluster=True, include_total=True)

# Thermal

In [ ]:
def preprocess_thermal(df):
    return df.groupby(['iteration', 'kernel', 'wa_path']).mean(numeric_only=True).reset_index()

def postprocess_thermal(df):
    for col in [c for c in df.columns if c not in ['time', 'iteration', 'kernel', 'wa_path']]:
        df[col] = df[col] / 1000
    df = round(df, 2)
    return df

gb5.load_combined_analysis('thermal.pqt', preprocess=preprocess_thermal, postprocess=postprocess_thermal)
gb5.analysis['thermal_melt'] = pd.melt(gb5.analysis['thermal'], id_vars=['iteration', 'wa_path', 'kernel'], value_vars=['little', 'mid', 'big']).rename(columns={'variable':'cluster'})
display(gb5.analysis['thermal'])

## Line plot

In [ ]:
ds = hv.Dataset(gb5.analysis['thermal_melt'], ['iteration', hv.Dimension('wa_path', values=gb5.wa_paths), hv.Dimension('cluster', values=gb5.CLUSTERS)], 'value')
layout = ds.to(hv.Curve, 'iteration', 'value').overlay('wa_path').opts(legend_position='bottom').layout('cluster').opts(title='Mean cluster temperature across iterations')
layout.opts(
    opts.Curve(width=600, height=600, ylabel='temperature'),
)
layout

## Bar plot

In [ ]:
gb5.summary['thermal'] = gb5.plot_gmean_bars(gb5.analysis['thermal_melt'], x='cluster', y='value', facet_col='metric', facet_col_wrap=2, title='Gmean temperature', width=1800, height=600, order_cluster=True, include_columns=['cluster'])

# Frequency

In [ ]:
def postprocess_freq(df):
    df['unit'] = 'MHz'
    df['metric'] = 'frequency'
    df['order'] = df['cluster'].replace('little', 0).replace('mid', 1).replace('big', 2)
    return df.sort_values(by=['iteration', 'order']).rename(columns={'frequency':'value'})

gb5.load_combined_analysis('freqs_mean.pqt', postprocess=postprocess_freq)
display(gb5.analysis['freqs_mean'].head())

## Line plot

In [ ]:
ds = hv.Dataset(gb5.analysis['freqs_mean'], ['iteration', hv.Dimension('wa_path', values=gb5.wa_paths), hv.Dimension('cluster', values=gb5.CLUSTERS)], 'value')
layout = ds.to(hv.Curve, 'iteration', 'value').overlay('wa_path').opts(legend_position='bottom').layout('cluster').opts(title='Mean cluster frequency across iterations')
layout.opts(
    opts.Curve(width=600, height=600, ylabel='MHz'),
)
layout

## Bar plot

In [ ]:
gb5.summary['frequency'] = gb5.plot_gmean_bars(gb5.analysis['freqs_mean'], x='metric', y='value', facet_col='cluster', facet_col_wrap=3, title='Gmean frequency per cluster', width=1800, height=600, order_cluster=True, include_columns=['cluster'])

# CFS signals

In [ ]:
gb5.load_combined_analysis('sched_pelt_cfs_mean.pqt')
gb5.analysis['sched_pelt_cfs_melt'] = pd.melt(gb5.analysis['sched_pelt_cfs_mean'], id_vars=['iteration', 'wa_path', 'kernel', 'cluster'], value_vars=['util', 'load'])

## Line plot

In [ ]:
signals = ['util', 'load']
ds = hv.Dataset(gb5.analysis['sched_pelt_cfs_mean'], ['iteration', hv.Dimension('wa_path', values=gb5.wa_paths), hv.Dimension('cluster', values=gb5.CLUSTERS)], signals)
layout = hv.Layout([ds.to(hv.Curve, 'iteration', signal).overlay('wa_path').opts(legend_position='bottom').layout('cluster').opts(title='Mean cluster ' + signal, framewise=True) for signal in signals]).cols(1)
layout.opts(
    opts.Curve(width=800, height=400, framewise=True),
)
layout

## Bar plot

In [ ]:
gb5.summary['cfs_signals'] = gb5.plot_gmean_bars(gb5.analysis['sched_pelt_cfs_melt'], x='cluster', y='value', facet_col='variable', facet_col_wrap=1, title='Gmean cfs signals', width=1900, height=1000, order_cluster=True, include_columns=['cluster'])

# Task wakeup latency

In [ ]:
def postprocess_wakeup_latency_mean(df):
    df = df.rename(columns={'wakeup_latency':'value'})
    df['order'] = df['wa_path'].map(lambda x: gb5.wa_paths.index(x))
    df['unit'] = 'x'
    return df

gb5.load_combined_analysis('wakeup_latency_mean.pqt', postprocess=postprocess_wakeup_latency_mean)

def postprocess_wakeup_latency(df):
    df['order'] = df['wa_path'].map(lambda x: gb5.wa_paths.index(x))
    df['cluster'] = df['cpu'].copy().apply(lambda c: 'little' if c < 4 else 'big' if c > 5 else 'mid')
    df['order_cluster'] = df['cluster'].map(lambda x: gb5.CLUSTERS.index(x))
    df['target_cluster'] = df['target_cpu'].copy().apply(lambda c: 'little' if c < 4 else 'big' if c > 5 else 'mid')
    df['order_target_cluster'] = df['target_cluster'].map(lambda x: gb5.CLUSTERS.index(x))
    return df

gb5.load_combined_analysis('wakeup_latency.pqt', postprocess=postprocess_wakeup_latency)
    
display(gb5.analysis['wakeup_latency'])

## Line plot

In [ ]:
gb5.plot_lines_px(gb5.analysis['wakeup_latency_mean'], facet_col='comm', facet_col_wrap=3, height=600, title='Task wakeup latencies across iterations')

## Bar plot

In [ ]:
gb5.summary['wakeup_latency'] = gb5.plot_gmean_bars(gb5.analysis['wakeup_latency_mean'], x='metric', y='value', facet_col='comm', facet_col_wrap=3, title='Gmean task wakeup latency', table_sort=['comm', 'kernel'], gmean_round=0, width=1900, height=600)

## Quantiles

In [ ]:
gb5.analysis['wakeup_latency_quantiles'] = gb5.analysis['wakeup_latency'].groupby(['comm', 'wa_path', 'iteration']).quantile([0.9, 0.95, 0.99], numeric_only=True).reset_index()[['comm', 'wa_path', 'level_3', 'iteration', 'wakeup_latency', 'order']].rename(columns={'level_3':'quantile'}).sort_values(by=['comm', 'order'])
gb5.analysis['wakeup_latency_quantiles']

In [ ]:
gb5.plot_gmean_bars(gb5.analysis['wakeup_latency_quantiles'].rename(columns={'wakeup_latency':'value'}), x='quantile', y='value', facet_col='comm', facet_col_wrap=1, title='Gmean latency quantile', width=1900, include_columns=['quantile'], table_sort=['quantile', 'comm'], height=1200, gmean_round=0)

## Per-execution cluster

In [ ]:
gb5.analysis['latency_cluster'] = gb5.analysis['wakeup_latency'].groupby(['comm', 'wa_path', 'cluster']).mean(numeric_only=True).reset_index().sort_values(by=['comm', 'order_cluster', 'order'])[['comm', 'wa_path', 'cluster', 'wakeup_latency']]
fig = px.bar(gb5.analysis['latency_cluster'], x='cluster', y='wakeup_latency', color='wa_path', facet_col='comm', barmode='group', facet_col_wrap=1, width=2000, height=1300, text=gb5.analysis['latency_cluster']['wakeup_latency'].apply(lambda x: trim_number(x)), title='Mean task wakeup latency per cluster')
fig.update_traces(textposition='outside')
gb5.analysis['latency_cluster']['wakeup_latency'] = gb5.analysis['latency_cluster']['wakeup_latency'].apply(lambda x: trim_number(x))
ptable(gb5.analysis['latency_cluster'])
fig.show(renderer='iframe')

## Per-target cluster

In [ ]:
gb5.analysis['latency_target_cluster'] = gb5.analysis['wakeup_latency'].groupby(['comm', 'wa_path', 'target_cluster']).mean(numeric_only=True).reset_index().sort_values(by=['comm', 'order_target_cluster', 'order'])[['comm', 'wa_path', 'target_cluster', 'wakeup_latency']]
fig = px.bar(gb5.analysis['latency_target_cluster'], x='target_cluster', y='wakeup_latency', color='wa_path', facet_col='comm', barmode='group', facet_col_wrap=1, width=2000, height=1300, text=gb5.analysis['latency_target_cluster']['wakeup_latency'].apply(lambda x: trim_number(x)), title='Mean task wakeup latency per target_cluster')
fig.update_traces(textposition='outside')
gb5.analysis['latency_target_cluster']['wakeup_latency'] = gb5.analysis['latency_target_cluster']['wakeup_latency'].apply(lambda x: trim_number(x))
ptable(gb5.analysis['latency_target_cluster'])
fig.show(renderer='iframe')

# Wakeup latency - cgroups

In [ ]:
def postprocess_cgroup_latency(df):
        df = df.rename(columns={'wakeup_latency':'value'})
        df['order'] = df['wa_path'].map(lambda x: gb5.wa_paths.index(x))
        return df

gb5.load_combined_analysis('wakeup_latency_cgroup.pqt', postprocess=postprocess_cgroup_latency)
gb5.analysis['wakeup_latency_cgroup_mean'] = gb5.analysis['wakeup_latency_cgroup'].groupby(["wa_path", "cgroup", "iteration", "order"]).mean(numeric_only=True).reset_index().sort_values(by=["order", "cgroup", "iteration"])[['wa_path', 'cgroup', 'iteration', 'value', 'order']]
gb5.analysis['wakeup_latency_cgroup_mean']

## Line plot

In [ ]:
gb5.plot_lines_px(gb5.analysis['wakeup_latency_cgroup_mean'], facet_col='cgroup', facet_col_wrap=3, height=600, title='cgroup wakeup latencies across iterations')

## Bar plot

In [ ]:
gb5.plot_gmean_bars(gb5.analysis['wakeup_latency_cgroup_mean'], x='metric', y='value', facet_col='cgroup', title='Gmean task wakeup latency per-cgroup', include_columns=['cgroup'], table_sort=['cgroup'], gmean_round=0, width=1800, height=600)

## Quantiles

In [ ]:
gb5.analysis['wakeup_latency_cgroup_quantiles'] = gb5.analysis['wakeup_latency_cgroup'].groupby(['cgroup', 'wa_path', 'iteration']).quantile([0.9, 0.95, 0.99], numeric_only=True).reset_index()[['cgroup', 'wa_path', 'level_3', 'iteration', 'value', 'order']].rename(columns={'level_3':'quantile'}).sort_values(by=['cgroup', 'order'])
gb5.analysis['wakeup_latency_cgroup_quantiles']

In [ ]:
gb5.plot_gmean_bars(gb5.analysis['wakeup_latency_cgroup_quantiles'], x='quantile', y='value', facet_col='cgroup', facet_col_wrap=1, title='Gmean latency quantile per-cgroup', include_columns=['cgroup', 'quantile'], table_sort=['quantile', 'cgroup'], width=1900, height=1400, gmean_round=0)

# Task CPU residency

In [ ]:
tasks_important = ['AsyncTask #1', 'labs.geekbench5', 'surfaceflinger']
cpus_prefix = [f"cpu{int(float(x))}" for x in gb5.CPUS]

def postprocess_tasks_residency_cpu_total(df):
    df = df.rename(columns={'Total':'total'})
    return df

gb5.load_combined_analysis('tasks_residency_cpu_total.pqt', postprocess=postprocess_tasks_residency_cpu_total)
gb5.analysis['tasks_residency_cpu_total_melt'] = pd.melt(gb5.analysis['tasks_residency_cpu_total'], id_vars=['iteration', 'wa_path', 'kernel'], value_vars=cpus_prefix).rename(columns={'variable':'cpu'}).sort_values(['wa_path', 'kernel', 'iteration', 'cpu'])
gb5.analysis['tasks_residency_cpu_total_cluster_melt'] = pd.melt(gb5.analysis['tasks_residency_cpu_total'], id_vars=['iteration', 'wa_path', 'kernel'], value_vars=gb5.CLUSTERS_TOTAL).rename(columns={'variable':'cluster'}).sort_values(['wa_path', 'kernel', 'iteration', 'cluster'])

def postprocess_tasks_residency_total(df):
    df = df.rename(columns={'Total':'total'}).query("comm in @tasks_important")
    return df

gb5.load_combined_analysis('tasks_residency_total.pqt', postprocess=postprocess_tasks_residency_total)
gb5.analysis['tasks_residency_total_melt'] = pd.melt(gb5.analysis['tasks_residency_total'], id_vars=['iteration', 'wa_path', 'kernel', 'comm'], value_vars=cpus_prefix).rename(columns={'variable':'cpu'}).sort_values(['wa_path', 'kernel', 'iteration', 'cpu'])
gb5.analysis['tasks_residency_total_cluster_melt'] = pd.melt(gb5.analysis['tasks_residency_total'], id_vars=['iteration', 'wa_path', 'kernel', 'comm'], value_vars=gb5.CLUSTERS_TOTAL).rename(columns={'variable':'cluster'}).sort_values(['wa_path', 'kernel', 'iteration', 'cluster'])

def postprocess_tasks_residency(df):
    df = df.rename(columns={'Total':'total'}).query("comm in @tasks_important")
    return df

gb5.load_combined_analysis('tasks_residency.pqt', postprocess=postprocess_tasks_residency)
gb5.analysis['tasks_residency_cluster_melt'] = pd.melt(gb5.analysis['tasks_residency'], id_vars=['iteration', 'wa_path', 'kernel', 'comm'], value_vars=gb5.CLUSTERS_TOTAL).rename(columns={'variable':'cluster'})

gb5.analysis['tasks_residency_total_melt']

## Clusters - Line plot

In [ ]:
ds = hv.Dataset(gb5.analysis['tasks_residency_cpu_total_cluster_melt'], ['iteration', hv.Dimension('wa_path', values=gb5.wa_paths), hv.Dimension('cluster', values=gb5.CLUSTERS_TOTAL)], 'value')
layout = ds.to(hv.Curve, 'iteration', 'value').overlay('wa_path').opts(legend_position='bottom').layout('cluster').cols(4).opts(title='Mean cluster CPU residency')
layout.opts(
    opts.Curve(width=500, height=600, ylabel='time'),
)
layout

## Clusters - Bar plot

In [ ]:
gb5.plot_gmean_bars(gb5.analysis['tasks_residency_cpu_total_cluster_melt'], x='cluster', y='value', facet_col='metric', facet_col_wrap=1, title='Gmean cluster CPU residency', include_columns=['cluster'], height=800, order_cluster=True, include_total=True)

## Clusters - Per-task bar plot

In [ ]:
gb5.summary['tasks_cpu_residency'] = gb5.plot_gmean_bars(gb5.analysis['tasks_residency_total_cluster_melt'], x='cluster', y='value', facet_col='comm', facet_col_wrap=1, title='Gmean cluster task CPU residency', include_columns=['cluster'], height=1200, order_cluster=True, include_total=True)

## CPUs - Line plot

In [ ]:
ds = hv.Dataset(gb5.analysis['tasks_residency_total_melt'], ['iteration', hv.Dimension('wa_path', values=gb5.wa_paths), hv.Dimension('cpu', values=cpus_prefix)], 'value')
layout = ds.to(hv.Curve, 'iteration', 'value').overlay('wa_path').opts(legend_position='bottom').layout('cpu').cols(3).opts(title='Mean CPU residency')
layout.opts(
    opts.Curve(width=600, height=600, ylabel='time'),
)
layout

## CPUs - Bar plot

In [ ]:
gb5.plot_gmean_bars(gb5.analysis['tasks_residency_total_melt'], x='cpu', y='value', facet_col='comm', facet_col_wrap=2, title='Gmean CPU residency', width=1900, height=700)

# CPU residency - cgroups

In [ ]:
def postprocess_cgroup_task_residency(df):
    df = df.rename(columns={'Total':'total'})[['wa_path', 'cgroup', 'iteration', 'total', 'little', 'mid', 'big'] + gb5.CPUS]
    return df

gb5.load_combined_analysis('tasks_residency_cgroup_total.pqt', postprocess=postprocess_cgroup_task_residency)
gb5.analysis['cgroup_residency_total_melt'] = pd.melt(gb5.analysis['tasks_residency_cgroup_total'], id_vars=['iteration', 'wa_path', 'cgroup'], value_vars=gb5.CPUS).rename(columns={'variable':'cpu'})
gb5.analysis['cgroup_residency_total_cluster_melt'] = pd.melt(gb5.analysis['tasks_residency_cgroup_total'], id_vars=['iteration', 'wa_path', 'cgroup'], value_vars=gb5.CLUSTERS_TOTAL).rename(columns={'variable':'cluster'})

display(gb5.analysis['cgroup_residency_total_melt'])

## Clusters - Bar plot

In [ ]:
gb5.plot_gmean_bars(gb5.analysis['cgroup_residency_total_cluster_melt'], x='cluster', y='value', facet_col='cgroup', facet_col_wrap=1, title='Gmean cluster CPU residency per-cgroup', width=1800, height=1100, include_columns=['cgroup', 'cluster'], table_sort=['cgroup', 'cluster'], order_cluster=True, include_total=True)

## CPUs - Bar plot

In [ ]:
gb5.plot_gmean_bars(gb5.analysis['cgroup_residency_total_melt'], x='cpu', y='value', facet_col='cgroup', facet_col_wrap=1, title='Gmean cgroup CPU residency', width=2000, height=1100, include_columns=['cgroup', 'cpu'])

# Summary - TLDR

In [ ]:
summary_parts = []

if 'scores' in gb5.summary:
    summary_benchmark_scores = gb5.summary['scores'].copy()
    summary_benchmark_scores['perc_diff'] = summary_benchmark_scores['perc_diff'].apply(lambda s: f"({s})")
    summary_benchmark_scores['value'] = summary_benchmark_scores['value'] + " " + summary_benchmark_scores['perc_diff']
    summary_benchmark_scores = summary_benchmark_scores.pivot(values='value', columns='kernel', index='metric').reset_index()[['metric'] + gb5.wa_paths]
    summary_parts.append(summary_benchmark_scores)

if 'power_usage' in gb5.summary:
    summary_power_usage = gb5.summary['power_usage'].copy().query("channel == 'CPU'")
    summary_power_usage['perc_diff'] = summary_power_usage['perc_diff'].apply(lambda s: f"({s})")
    summary_power_usage['value'] = summary_power_usage['value'] + " " + summary_power_usage['perc_diff']
    summary_power_usage['channel'] = 'CPU_total_power'
    summary_power_usage = summary_power_usage.pivot(values='value', columns='kernel', index='channel').reset_index().rename(columns={'channel':'metric'})[['metric'] + gb5.wa_paths]
    summary_parts.append(summary_power_usage)

if 'overutilized_mean' in gb5.analysis:
    summary_ou = gb5.analysis['overutilized_mean'].copy()
    summary_ou['percentage'] = summary_ou['percentage'].apply(lambda x: f"{x}%")
    summary_ou = summary_ou.pivot(values='percentage', columns='wa_path', index='metric').reset_index()[['metric'] + gb5.wa_paths]
    summary_parts.append(summary_ou)

if 'thermal' in gb5.summary:
    summary_thermal = gb5.summary['thermal'].copy()
    summary_thermal['perc_diff'] = summary_thermal['perc_diff'].apply(lambda s: f"({s})")
    summary_thermal['value'] = summary_thermal['value'] + " " + summary_thermal['perc_diff']
    summary_thermal = summary_thermal.pivot(values='value', columns='kernel', index='cluster').reset_index().rename(columns={'cluster':'metric'})[['metric'] + gb5.wa_paths]
    summary_thermal['metric'] = "thermal (" + summary_thermal['metric'] + ")"
    summary_parts.append(summary_thermal)

if 'wakeup_latency' in gb5.summary:
    summary_wakeup_latency = gb5.summary['wakeup_latency'].copy()
    summary_wakeup_latency['perc_diff'] = summary_wakeup_latency['perc_diff'].apply(lambda s: f"({s})")
    summary_wakeup_latency['comm'] = "latency (" + summary_wakeup_latency['comm'] + ")"
    summary_wakeup_latency['value'] = summary_wakeup_latency['value'] + " " + summary_wakeup_latency['perc_diff']
    summary_wakeup_latency = summary_wakeup_latency.pivot(values='value', columns='kernel', index='comm').reset_index().rename(columns={'comm':'metric'})[['metric'] + gb5.wa_paths]
    summary_parts.append(summary_wakeup_latency)

if 'task_cpu_residency' in gb5.summary:
    summary_task_cpu_residency = gb5.summary['task_cpu_residency'].copy().query("cluster == 'total'")
    summary_task_cpu_residency['perc_diff'] = summary_task_cpu_residency['perc_diff'].apply(lambda s: f"({s})")
    summary_task_cpu_residency['value'] = summary_task_cpu_residency['value'] + " " + summary_task_cpu_residency['perc_diff']
    summary_task_cpu_residency = summary_task_cpu_residency.pivot(values='value', columns='kernel', index='comm').reset_index().rename(columns={'comm':'metric'})[['metric'] + gb5.wa_paths]
    summary_task_cpu_residency['metric'] = "CPU residency (" + summary_task_cpu_residency['metric'] + ")"
    summary_parts.append(summary_task_cpu_residency)


summary = pd.concat(summary_parts).reset_index(drop=True)

print('Geekbench 5')
ptable(summary)